In [3]:
import transformers
import torch
from transformers import pipeline
from accelerate import Accelerator

accelerator = Accelerator()
device = accelerator.device
    
model = transformers.T5ForConditionalGeneration.from_pretrained("google/flan-t5-xxl", torch_dtype=torch.bfloat16, cache_dir="/mnt/disk-1/flan-t5-xxl")         
tokenizer = transformers.AutoTokenizer.from_pretrained("google/flan-t5-xxl")
        
model, tokenizer = accelerator.prepare(model, tokenizer)

In [5]:
batches = ["Answer using only one word - Alligator or Spanner and not Shovel. Which is more similar in meaning to Shovel?",
"Answer using only one word - Caiman or Tortoise and not Anvil. Which is more similar in meaning to Anvil?"]

batches = [b + " Let's think step by step." for b in batches]

inputs = tokenizer(batches, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs = {k: v.to(device) for k, v in inputs.items()}

outputs = model.generate(**inputs, max_length=512, do_sample=False)

responses = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


prompt_and_response = list(zip(batches, responses))

# prompt_and_response = []
# for prompt, response in zip(batches, responses):
#     prompt_and_response.append([prompt, response])
    
prompt_and_response

[("Answer using only one word - Alligator or Spanner and not Shovel. Which is more similar in meaning to Shovel? Let's think step by step.",
  'Alligator is a reptile. Spanner is a tool. So, the answer is Alligator.'),
 ("Answer using only one word - Caiman or Tortoise and not Anvil. Which is more similar in meaning to Anvil? Let's think step by step.",
  'Anvil is a heavy object used for hammering. Caiman is a reptile. Tortoise is a reptile.')]

In [ ]:
import csv
def save_responses(reponses, file_path):
    """the helper function for saving the responses"""
    
    with open(file_path, 'w') as f:
        # using csv.writer method from CSV package
        write = csv.writer(f)
        write.writerows(reponses)
        
save_responses(prompt_and_response, "test.csv")